# Experimental Methods
In the week 1 lab, we asked you to train and test a logistic regression model. We did this by splitting data into a training and test set. In this tutorial, we explore this concept a little further, and introduce you to other approaches for training (and testing) machine learning models.

For this tutorial, we will be using the Iris dataset, which is included with the scikit library. The dataset is 

In [ ]:
from sklearn import datasets
import numpy as np
iris = datasets.load_iris()
X = iris.data #these are the features
y = iris.target # these are the labels

We will only use three of the four features in this tutorial.

In [ ]:
X = X[:,[0,1,2]]

As a reminder, when we train a machine learning model, we ALWAYS need to test the resulting model on some data that was not used in the training. In the course handbook, we drew an analogy to revising for an exam. Training is like revising for an exam using past papers, but we can only test your performance properly by using a new exam paper.

The simplest way to do this is to obtain two separate data sets - one for training, and one for testing. However, in most cases, we only have one data set. In this case, we have a few different options, which we go through now

## train/test split
The first option is to split our single data set into a training and test set. In this scheme, we randomly select examples for the each of the data sets, without replacement. It is important that examples are chosen at *random* because in real-life problems, we cannot assume that the data set is random to begin with (e.g. they might be sorted by the output label).

scikit gives us an easy function for doing this:

In [ ]:
# train/test split
from sklearn.model_selection import train_test_split
print("total examples = " + str(len(X)))
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=41)
print("total training example = " + str(len(X_train)))

We can then train any model on the training set. For instance, let's try a logistic regression:

In [ ]:
# train a logistic regression
from sklearn.linear_model import LogisticRegression

# create model
model = LogisticRegression(max_iter = 200)

#train model
model.fit(X_train, y_train)

We report the performace of our model on the test set. We can view how well a model does using a confusion matrix. 

In [ ]:
#confusion matrix
y_pred = model.predict(X_test) # use the trained model on the test data

from sklearn.metrics import confusion_matrix
c_matrix = confusion_matrix(y_test, y_pred)

We can summarise a models performance using its *accuracy*. Wed define accuracy as the fraction of classifications that correct, over the total number of samples. 
In Chapter 8 of the course handbook, we will introduce you to two further common measures of model performance, sensitivity and specificity. In the code snippet below, we calculate these three metrics this manually, and show that it is equivalent to the in-built sklearn function.

In [ ]:
# report accuracy
TP = 0 #true positives
total = 0
for i in range(0, 3):
    TP = TP + c_matrix[i][i]
    total = total + sum(c_matrix[i])
    
acc = TP / total
print("calculated accuracy = " + str(acc))

from sklearn.metrics import accuracy_score
print("sklearn accuracy = " + str(accuracy_score(y_test, y_pred)))

Similarly, we can calculate other common metrics such as sensitivity and specificity from the confusion matrix or using the relevant scikit functions as shown below

In [ ]:
# we can do all of this in one go using the classification_report function
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

How do we select what percentage of the data set to keep for testing? There is no fixed rule for this, and it depends on the size of the data set. You should have sufficient examples in the training set such that the training set is 'representative' of the whole data set, but enough testing examples so that you can get a reasonable assessment of performance.

For instance, if we consider an extreme case, where there are 100 examples and 99 are used for training. The training set is then *very* representative of the whole data set, but our single test example means that we can only be 100% or 0% accurate. A common ratio for train:test is 2:1, and it is common to see greater ratio (e.g. 4:1) for data sets with many examples.

## k-fold cross validation
Doing a train/test split is fine, but it is subject to how *good* the split is. For instance, we might get very lucky, such that the training set, by chance, contains only the examples that are very easy to classify.

To reduce the impact of chance, we can employ k-fold cross validation. In this scheme, we split the whole data set into k equal parts. k-1 parts are used as training data, and the remaining part is used as test data (so far, this is equivalent to the test/train method with and 80/20 split). We calculate our performance metrics as usual.

We then repeat the process k times, using a different k-1 parts for training each time. By doing so, we end up with k different estimates for the performance metrics, which we can average to get an overall estimate of the model performance. We can also calculate the standard deviation of the k estimates to get an idea of the stability of the model (i.e. how sensitive is it to slightly different data from the same data set?).

scikit has a built in method for this, which we run below

In [ ]:
#cross validation
from sklearn.model_selection import cross_val_score
# re-initialise the model, as we are need the UNTRAINED model
model = LogisticRegression(max_iter = 200)
cross_val_score(model, X, y, cv=5)

## leave one out cross-validation
Leave one out cross-validation is standard k-fold cross validation taken to the extreme, in which the test set is just one example. It is useful when we have very small data sets. It involves creating N models, where N is the total number of examples. It is thus computationally expensive, hence why it is only really used for small data sets.

In [ ]:
#LOOCV
from sklearn.model_selection import LeaveOneOut
loocv = LeaveOneOut()
model = LogisticRegression(max_iter = 200)
results_loocv = cross_val_score(model, X, y, cv=loocv)
print("Accuracy: %.2f%%" % (results_loocv.mean()*100.0))

## train/validate/test
For some of the machine learning models you will encounter later in this course, we have the option of adjusting some free parameters - known as hyperparameters. We generally adjust these so that they give the best result on on some test data. In other words, we are learning the best value for the hyperparameters using the test data. Adjusting the hyperparameters is *itself* part of model training.

This means that, for these types of problems, we need to split the data into three parts: training data (to learn the model parameters), validation data (to learn the hyperparameters, and test data (to test the fully trained model).

Below, we show how the train_test_split function can be used twice to achieve this.

In [ ]:
# train/validate/test - split into equal thirds
X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.33)
X_train, X_valid, y_train, y_valid = train_test_split(X_temp, y_temp, test_size=0.5)

print ("examples in training set: " + str(len(X_train)))
print ("examples in validation set: " + str(len(X_valid)))
print ("examples in test set: " + str(len(X_test)))

We can extend k-fold cross validation in the same way. We keep a test set separate for final testing (using, say 20% of the data), and then we perform k-fold cross-validation on the rest of the remaining portion of the data set.

In [ ]:
# cross validation with testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

model = LogisticRegression(max_iter = 200)

# initialise an array to store accuracies for each model
model_acc = [None] * 4

# in real use, dummy_parameter would be some hyperparameter that affects the model.
# you will come across some examples of these next week when we look at SVMs.
for dummy_parameter in range(0,4):
    scores = cross_val_score(model, X, y, cv=5)
    model_acc[dummy_parameter] = scores.mean()
    
# The hyperparameters with best model_acc can then be tested on X_test, y_test

This is the end of the tutorial on experimental methods. Next week, you will encounter data normalisation/standardisation. You may wish to revisit this notebook then, to think about where and when it is appropriate to normalize your data.